In [46]:
from classiq import *
from classiq.execution import ExecutionPreferences

# Quantum walk on a line
We want to perform a quantum walk on a line with 16 nodes:
#### 0--1--2--3--...--14--15
To perform the walk, we move from the current node to one of the adjecent nodes with equal probability. Notice that at the ends of the line we only have one adjecent node, therefore only one option to move.
We require two components: the Coin operator C which determines the next destination of the walker, and the Shift operator which "shift" the walker to the next destination.

#### Coin operator

In [29]:
@qfunc
def prepare_minus(x: QBit):
  '''
  Prepare the qubit x in the minus state.
  '''
  X(x)
  H(x)

In [30]:
@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  '''
  Perform the diffuzer oracle: check whether the qnum x is zero, and save the result in the aux qnum.
  '''
  aux^=(x!=0)

In [31]:
@qfunc
def zero_diffuzer(x: QNum):
  '''
  Perform the zero diffuzer: reflect the qnum x around zero. If x==0, then it's not affected, otherwise picks up a minus sign.
  '''
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)

In [32]:
def C_iteration(i:int, vertices: QNum, adjacent_vertices:QNum):
    '''
    Perform the Coin operation at the current node i.
    '''
    prob = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    if i==0: # we can't loop around when we are at the edges of the line 
        prob[1]=1.
    elif i==15:
        prob[14]=1.
    else:
      prob[(i+1)]=0.5
      prob[(i-1)]=0.5
    print(f'State={i}, prob vec ={prob}')
    
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))


In [33]:
@qfunc 
def C_operator(vertices:QNum, adjacent_vertices: QNum):
    '''
    Perform the Coin operation at all vertices.
    '''
    for i in range(2**size):
      C_iteration(i,vertices,adjacent_vertices)

#### Shift operator

In [52]:
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  '''
  Check if the edge exists. i.e. whether two verties are adjecent.
  '''
  distance = vertices-adjacent_vertices
  res |= ((distance == 1) | (distance == -1)) # abs() is not currently in supported functions

In [53]:
@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  '''
  Bitwise swap of the values x and y.
  '''
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))

In [54]:
@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    '''
    Perform the Shift operator: move from the current vertex onto the adjecent vertices.
    '''
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

#### Quantum walk

In [55]:
@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
  '''
  Perform the quantum walk over the vertices by applying the Coin operator (to find the next nodes), followed by 
  the Shift operator (to perform the movement onto the next nodes).
  '''

  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size, adjacent_vertices)

  C_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

In [58]:
size = 4 # we need 4 qubits to encode 16 nodes
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0